### Data Collection

In [16]:
import tarfile
import os

def extract_files(filename, extract_path):
    # Check if extraction is necessary
    if os.path.exists(extract_path):
        print(f"Files already extracted in {extract_path}. Skipping extraction.")
        return

    # Check if the file is a tar archive
    if filename.endswith('.tar') or filename.endswith('.tgz'):
        with tarfile.open(filename, 'r:*') as tar:
            tar.extractall(extract_path)
            print(f"Extracted {filename} to {extract_path}")
    else:
        print("File is not a tar or tgz archive.")

if __name__ == '__main__':
    # Extract words.tgz
    extract_files('data/words.tgz', 'data/words')

    # Extract xml.tar
    extract_files('data/xml.tar', 'data/xml')


Files already extracted in data/words. Skipping extraction.
Files already extracted in data/xml. Skipping extraction.
